## Classification
Put simply, classification is the task of predicting a label for a given observation. For example: you are given certain physical descriptions of an animal, and your taks is to classify them as either a dog or a cat. Here, we will classify iris flowers.

As we will see later, we will use different classifiers and at the end of this notebook, we will compare them. We will define our accuracy function right now to get it out of the way. We will use a simple accuracy function that returns the ratio of the number of correctly classified observations to the total number of predictions.

In [15]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [1]:
using Pkg
Pkg.add("GLMNet")
Pkg.add("DecisionTree")
Pkg.add("NearestNeighbors")
Pkg.add("Random")
Pkg.add("DataStructures")
Pkg.add("LIBSVM")

    Updating registry at `~/.julia/registries/General`
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
   Resolving package versions...
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/s

In [2]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

Get the data first

In [3]:
iris = dataset("datasets", "iris")

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [4]:
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]

150-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [5]:
X

150×4 Matrix{Float64}:
 5.1  3.5  1.4  0.2
 4.9  3.0  1.4  0.2
 4.7  3.2  1.3  0.2
 4.6  3.1  1.5  0.2
 5.0  3.6  1.4  0.2
 5.4  3.9  1.7  0.4
 4.6  3.4  1.4  0.3
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.9  3.1  1.5  0.1
 5.4  3.7  1.5  0.2
 4.8  3.4  1.6  0.2
 4.8  3.0  1.4  0.1
 ⋮              
 6.0  3.0  4.8  1.8
 6.9  3.1  5.4  2.1
 6.7  3.1  5.6  2.4
 6.9  3.1  5.1  2.3
 5.8  2.7  5.1  1.9
 6.8  3.2  5.9  2.3
 6.7  3.3  5.7  2.5
 6.7  3.0  5.2  2.3
 6.3  2.5  5.0  1.9
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3
 5.9  3.0  5.1  1.8

In [6]:
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)

150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In classification, we often want to use some of the data to fit a model, and the rest of the data to validate (commonly known as `training` and `testing` data). We will get this data ready now so that we can easily use it in the rest of this notebook.

In [7]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [8]:
?randsubseq

search: randsubseq randsubseq! RandomSub StratifiedRandomSub



```
randsubseq([rng=GLOBAL_RNG,] A, p) -> Vector
```

Return a vector consisting of a random subsequence of the given array `A`, where each element of `A` is included (in order) with independent probability `p`. (Complexity is linear in `p*length(A)`, so this function is efficient even if `p` is small and `A` is large.) Technically, this process is known as "Bernoulli sampling" of `A`.

# Examples

```jldoctest
julia> rng = MersenneTwister(1234);

julia> randsubseq(rng, 1:8, 0.3)
2-element Vector{Int64}:
 7
 8
```


In [9]:
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

42-element Vector{Int64}:
   5
   6
   9
  27
  33
  34
  41
  42
  47
  50
  51
  55
  56
   ⋮
 112
 115
 124
 125
 127
 129
 132
 136
 138
 140
 142
 145

We will need one more function, and that is the function that will assign classes based on the predicted values when the predicted values are continuous.

In [10]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

### 🟣 Method 1: Lasso

In [11]:
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])

Least Squares GLMNet Cross Validation
73 models for 4 predictors in 10 folds
Best λ 0.011 (mean loss 0.056, std 0.007)

In [12]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])
mylambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda]);

In [13]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)

42×1 Matrix{Float64}:
 0.927631094189582
 1.0642130476063296
 1.0141568866944402
 1.1103294156005634
 0.8182141624997881
 0.853466129185418
 0.9876047158766
 1.135934645884928
 0.9342849782066667
 0.9647135766916641
 2.2543775623438793
 2.351433666532979
 2.2091447696326227
 ⋮
 2.7268111353805686
 2.9995775873392034
 2.6007594936480634
 2.8419983758044705
 2.572710896638133
 2.8881147437987043
 2.826708061170094
 3.068433063734618
 2.645442800700705
 2.81965672227815
 2.8991945411415614
 3.095200630586712

In [16]:
predictions_lasso = assign_class.(predictions_lasso)
findaccuracy(predictions_lasso,y[testids])

0.9761904761904762

### 🟣 Method 2: Ridge
We will use the same function but set alpha to zero.

In [17]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

0.9761904761904762

### 🟣 Method 3: Elastic Net
We will use the same function but set alpha to 0.5 (it's the combination of lasso and ridge).

In [18]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

0.9761904761904762

### 🟣 Method 4: Decision Trees
We will use the package `DecisionTree`

In [19]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [20]:
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

0.9761904761904762

### 🟣 Method 5: Random Forests
The `RandomForestClassifier` is available through the `DecisionTree` package as well.

In [21]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 6.6
Avg Depth:  4.75

In [22]:
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

0.9761904761904762

### 🟣 Method 6: Using a Nearest Neighbor method
We will use the `NearestNeighbors` package here.

In [23]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

KDTree{StaticArrays.SVector{4, Float64}, Euclidean, Float64}
  Number of points: 108
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true

In [24]:
queries = X[testids,:]

42×4 Matrix{Float64}:
 5.0  3.6  1.4  0.2
 5.4  3.9  1.7  0.4
 4.4  2.9  1.4  0.2
 5.0  3.4  1.6  0.4
 5.2  4.1  1.5  0.1
 5.5  4.2  1.4  0.2
 5.0  3.5  1.3  0.3
 4.5  2.3  1.3  0.3
 5.1  3.8  1.6  0.2
 5.0  3.3  1.4  0.2
 7.0  3.2  4.7  1.4
 6.5  2.8  4.6  1.5
 5.7  2.8  4.5  1.3
 ⋮              
 6.4  2.7  5.3  1.9
 5.8  2.8  5.1  2.4
 6.3  2.7  4.9  1.8
 6.7  3.3  5.7  2.1
 6.2  2.8  4.8  1.8
 6.4  2.8  5.6  2.1
 7.9  3.8  6.4  2.0
 7.7  3.0  6.1  2.3
 6.4  3.1  5.5  1.8
 6.9  3.1  5.4  2.1
 6.9  3.1  5.1  2.3
 6.7  3.3  5.7  2.5

In [25]:
idxs, dists = knn(kdtree, queries', 5, true)

([[32, 1, 15, 6, 34], [16, 8, 40, 37, 17], [33, 4, 35, 11, 39], [21, 36, 6, 34, 15], [17, 40, 8, 14, 19], [13, 14, 12, 8, 40], [15, 1, 6, 32, 34], [33, 38, 11, 10, 2], [17, 19, 40, 24, 8], [6, 34, 30, 1, 25]  …  [105, 93, 53, 97, 62], [89, 103, 101, 82, 85], [93, 100, 105, 62, 97], [96, 77, 85, 82, 106], [86, 78, 80, 92, 91], [95, 78, 76, 91, 92], [85, 77, 106, 81, 82], [82, 104, 89, 101, 106], [104, 82, 81, 106, 84], [101, 89, 103, 99, 82]], [[0.14142135623730925, 0.1414213562373093, 0.17320508075688756, 0.22360679774997916, 0.24494897427831783], [0.33166247903553986, 0.3464101615137753, 0.3605551275463989, 0.3741657386773947, 0.38729833462074226], [0.14142135623730948, 0.29999999999999966, 0.31622776601683816, 0.34641016151377546, 0.36055512754639873], [0.1999999999999998, 0.22360679774997896, 0.22360679774997902, 0.24494897427831772, 0.2645751311064591], [0.3741657386773941, 0.42426406871192796, 0.4582575694955836, 0.458257569495584, 0.5099019513592782], [0.3605551275463992, 0.38729

In [26]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(string(argmax(possible_labels[i])))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

0.9761904761904762

### 🟣 Method 7: Support Vector Machines
We will use the `LIBSVM` package here.

In [27]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

108-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [28]:
model = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Vector{Int64}, Matrix{Float64}}(38, Int32[5, 16, 17], [1, 1, 1, 1, 1, 2, 2, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [4.3 5.7 … 6.2 5.9; 3.0 4.4 … 3.4 3.0; 1.1 1.5 … 5.4 5.1; 0.1 0.4 … 2.3 1.8], Int32[11, 13, 21, 22, 23, 42, 44, 50, 52, 53  …  93, 94, 97, 98, 100, 102, 105, 106, 107, 108], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 4.3), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 4.8), LIBSVM.SVMNode(1, 5.0), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.2), LIBSVM.SVMNode(1, 5.9), LIBSVM.SVMNode(1, 6.3)  …  LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 7.2), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 6.0), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 6.2), LIBSVM.SVMNode(1, 5.9)]), 0.0, [0.2654865893543365 0.839698554488715; 0.4440322328728615 0.85632893

In [29]:
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

0.9761904761904762

Putting all the results together:

In [30]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso,ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge,ytest)
overall_accuracies[3] = findaccuracy(predictions_EN,ytest)
overall_accuracies[4] = findaccuracy(predictions_DT,ytest)
overall_accuracies[5] = findaccuracy(predictions_RF,ytest)
overall_accuracies[6] = findaccuracy(predictions_NN,ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM,ytest)
hcat(methods, overall_accuracies)

7×2 Matrix{Any}:
 "lasso"  0.97619
 "ridge"  0.97619
 "EN"     0.97619
 "DT"     0.97619
 "RF"     0.97619
 "kNN"    0.97619
 "SVM"    0.97619

# Finally...
After finishing this notebook, you should be able to:
- [x] split your data into training and testing data to test the effectiveness of a certain method
- [x] apply a simple accuracy function to test the effectiveness of a certain method
- [x] run multiple classification algorithms:
    - [x] LASSO
    - [x] Ridge
    - [x] ElasticNet
    - [x] Decision Tree
    - [x] Random Forest
    - [x] Nearest Neighbors
    - [x] Support Vector Machines

# 🥳 One cool finding

We used multiple methods to run classification on the `iris` dataset which is a dataset of flowers and there are three types of iris flowers in it. We split the data into training and testing and ran our methods. Here is the scoreboard:

| method | accuracy score |
|---|---|
| lasso  |0.97619|
| ridge  |0.97619|
| EN     |0.07619|
| DT     |0.07619|
| RF     |0.07619|
| kNN    |0.07619|
| SVM    |0.07619|